In [1]:
import pandas as pd 
import numpy as np
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)

In [3]:
spam_df = pd.read_csv(r"spam.csv",encoding="ISO-8859-1")

In [44]:
spam_df.shape

(5572, 5)

In [45]:
spam_df.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [46]:
spam_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [47]:
spam_df = spam_df.dropna(axis=1,thresh=500)

In [48]:
spam_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [49]:
spam_df.columns = ["Prediction","Message"]

In [50]:
spam_df.head()

,Prediction,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [51]:
spam_df["Prediction"].value_counts()

ham     4825
spam     747
Name: Prediction, dtype: int64

# Preprocessor

In [11]:
class Preprocessor:
    def column_sanitizer(self,column):
        for i in range(len(column)): # Looping through the length of the array to make necessary changes
            punct_removal = column[i].translate(str.maketrans("","",punctuation)).lower() # Removed punctuations and converted to lower case
            filtered_words = [filt for filt in punct_removal.split() if filt not in stopwords.words("english") and len(filt) > 3] # Selected only words greater than 3 not in stopwords
            column[i] = " ".join(filtered_words) # Setting the index of the array to the preprocessed output
        return pd.Series(column) # Converting the output to series after all changes has been made
    def lemmatize(self, column):
        Lemmatizer = WordNetLemmatizer()
        pos_tag = {
            "N":wordnet.NOUN,
            "A":wordnet.ADJ,    # Created a dictionary 
            "R":wordnet.ADV,
            "V":wordnet.VERB
        }
        # nltk.pos_tag returns a list of tuple containing the word and the part of speech it belongs to
        #nltk.pos_tag takes in a list of string
        Pos_tag = lambda x: pos_tag.get(nltk.pos_tag([x])[0][1][0].lower(),wordnet.NOUN) # Created a function that gets the part of speech of a word  
        
        for i in range(len(column)):
            # Lemmatizer works well when the word and the part of speech is passed in as arguments
            lemmatized_str = "".join([Lemmatizer.lemmatize(word,Pos_tag(word)) for word in column[i]]) # Lemmatizing each words in the sentence
            column[i] = lemmatized_str # Setting the index of the array to the preprocessed output
        return pd.Series(column) # Converting the output to series after all changes has been made

In [12]:
preprocessor = Preprocessor()

In [13]:
spam_df["Message"] = preprocessor.column_sanitizer(spam_df["Message"])

In [14]:
spam_df["Message"] = preprocessor.lemmatize(spam_df["Message"])

In [15]:
spam_df.head()

,Prediction,Message
0,ham,jurong point crazy available bugis great world...
1,ham,joking
2,spam,free entry wkly comp final tkts 21st 2005 text...
3,ham,early already
4,ham,dont think goes lives around though


In [16]:
spam_df.isnull().sum()

Prediction    0
Message       0
dtype: int64

In [17]:
spam_df.shape

(5572, 2)

# Encoding target feature

In [18]:
spam_df["Prediction"] = LabelEncoder().fit_transform(spam_df[["Prediction"]])

In [19]:
spam_df.head()

,Prediction,Message
0,0,jurong point crazy available bugis great world...
1,0,joking
2,1,free entry wkly comp final tkts 21st 2005 text...
3,0,early already
4,0,dont think goes lives around though


In [20]:
target = spam_df["Prediction"]

In [21]:
sms= spam_df["Message"]

In [22]:
sms.shape

(5572,)

# TF-IDF vectorizer

In [23]:
sms = TfidfVectorizer().fit_transform(sms).toarray()

In [24]:
sms

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
sms.shape

(5572, 8282)

# Stratified KFold split

In [27]:
fold = StratifiedKFold(n_splits=15)

# Multinomial Naive-Bayes

In [28]:
multinomial_bayes = MNB(alpha=0.6)

In [55]:
def model_evaluaton(sms, target, model):
    train_acc_score = []
    test_acc_score = []
    for (train_split, test_split) in fold.split(sms, target):
        xtrain, xtest = sms[train_split], sms[test_split]
        ytrain, ytest = target[train_split], target[test_split]
        model.fit(xtrain,ytrain)
        train_pred = model.predict(xtrain)
        test_pred = model.predict(xtest)
        train_acc_score.append(accuracy_score(train_pred, ytrain))
        test_acc_score.append(accuracy_score(test_pred, ytest))
    print(f"\nAccuracy score for training data: {np.mean(train_acc_score)}\n\nAccuracy score for validation data: {np.mean(test_acc_score)}")

In [35]:
model_evaluaton(sms=sms,target=target, model=multinomial_bayes)

Accuracy score for training data: 0.986924420353088

Accuracy score for validation data: 0.9748765807804155


# Catboost 

In [37]:
catboost_ = CatBoostClassifier() 

In [38]:
model_evaluaton(sms=sms,target=target, model=catboost_)

Learning rate set to 0.020829
0:	learn: 0.6705151	total: 261ms	remaining: 4m 20s
1:	learn: 0.6501313	total: 335ms	remaining: 2m 47s
2:	learn: 0.6303269	total: 414ms	remaining: 2m 17s
3:	learn: 0.6119858	total: 490ms	remaining: 2m 2s
4:	learn: 0.5933960	total: 565ms	remaining: 1m 52s
5:	learn: 0.5748708	total: 632ms	remaining: 1m 44s
6:	learn: 0.5593035	total: 698ms	remaining: 1m 38s
7:	learn: 0.5442579	total: 778ms	remaining: 1m 36s
8:	learn: 0.5291765	total: 845ms	remaining: 1m 33s
9:	learn: 0.5141282	total: 914ms	remaining: 1m 30s
10:	learn: 0.4995821	total: 984ms	remaining: 1m 28s
11:	learn: 0.4863918	total: 1.06s	remaining: 1m 27s
12:	learn: 0.4746355	total: 1.13s	remaining: 1m 25s
13:	learn: 0.4623077	total: 1.23s	remaining: 1m 26s
14:	learn: 0.4511735	total: 1.3s	remaining: 1m 25s
15:	learn: 0.4408966	total: 1.38s	remaining: 1m 24s
16:	learn: 0.4317049	total: 1.46s	remaining: 1m 24s
17:	learn: 0.4226624	total: 1.54s	remaining: 1m 23s
18:	learn: 0.4155414	total: 1.62s	remaining: 1

158:	learn: 0.1588568	total: 13.5s	remaining: 1m 11s
159:	learn: 0.1585353	total: 13.6s	remaining: 1m 11s
160:	learn: 0.1581803	total: 13.7s	remaining: 1m 11s
161:	learn: 0.1577568	total: 13.8s	remaining: 1m 11s
162:	learn: 0.1574018	total: 13.9s	remaining: 1m 11s
163:	learn: 0.1570487	total: 14s	remaining: 1m 11s
164:	learn: 0.1567365	total: 14.1s	remaining: 1m 11s
165:	learn: 0.1563865	total: 14.2s	remaining: 1m 11s
166:	learn: 0.1561296	total: 14.2s	remaining: 1m 11s
167:	learn: 0.1556652	total: 14.3s	remaining: 1m 10s
168:	learn: 0.1552833	total: 14.4s	remaining: 1m 10s
169:	learn: 0.1549115	total: 14.5s	remaining: 1m 10s
170:	learn: 0.1545085	total: 14.5s	remaining: 1m 10s
171:	learn: 0.1540482	total: 14.6s	remaining: 1m 10s
172:	learn: 0.1537083	total: 14.7s	remaining: 1m 10s
173:	learn: 0.1533445	total: 14.8s	remaining: 1m 10s
174:	learn: 0.1531005	total: 14.9s	remaining: 1m 10s
175:	learn: 0.1527119	total: 14.9s	remaining: 1m 9s
176:	learn: 0.1524706	total: 15s	remaining: 1m 9s

317:	learn: 0.1216388	total: 25.6s	remaining: 54.8s
318:	learn: 0.1214469	total: 25.6s	remaining: 54.7s
319:	learn: 0.1212952	total: 25.7s	remaining: 54.6s
320:	learn: 0.1211960	total: 25.8s	remaining: 54.5s
321:	learn: 0.1209920	total: 25.8s	remaining: 54.4s
322:	learn: 0.1208406	total: 25.9s	remaining: 54.3s
323:	learn: 0.1207422	total: 26s	remaining: 54.2s
324:	learn: 0.1203820	total: 26.1s	remaining: 54.1s
325:	learn: 0.1201919	total: 26.1s	remaining: 54s
326:	learn: 0.1200847	total: 26.2s	remaining: 54s
327:	learn: 0.1197977	total: 26.3s	remaining: 53.9s
328:	learn: 0.1196245	total: 26.4s	remaining: 53.8s
329:	learn: 0.1194900	total: 26.4s	remaining: 53.7s
330:	learn: 0.1194024	total: 26.5s	remaining: 53.6s
331:	learn: 0.1191909	total: 26.6s	remaining: 53.5s
332:	learn: 0.1189820	total: 26.6s	remaining: 53.4s
333:	learn: 0.1188149	total: 26.7s	remaining: 53.3s
334:	learn: 0.1186746	total: 26.8s	remaining: 53.2s
335:	learn: 0.1185543	total: 26.8s	remaining: 53s
336:	learn: 0.118310

476:	learn: 0.1016618	total: 38.1s	remaining: 41.7s
477:	learn: 0.1015167	total: 38.1s	remaining: 41.7s
478:	learn: 0.1014612	total: 38.2s	remaining: 41.6s
479:	learn: 0.1012805	total: 38.3s	remaining: 41.5s
480:	learn: 0.1012214	total: 38.4s	remaining: 41.4s
481:	learn: 0.1011664	total: 38.5s	remaining: 41.3s
482:	learn: 0.1011124	total: 38.5s	remaining: 41.3s
483:	learn: 0.1010578	total: 38.6s	remaining: 41.2s
484:	learn: 0.1009989	total: 38.7s	remaining: 41.1s
485:	learn: 0.1009454	total: 38.8s	remaining: 41s
486:	learn: 0.1008923	total: 38.8s	remaining: 40.9s
487:	learn: 0.1006651	total: 38.9s	remaining: 40.8s
488:	learn: 0.1004033	total: 39s	remaining: 40.7s
489:	learn: 0.1001900	total: 39s	remaining: 40.6s
490:	learn: 0.1001324	total: 39.1s	remaining: 40.5s
491:	learn: 0.1000788	total: 39.2s	remaining: 40.5s
492:	learn: 0.1000204	total: 39.3s	remaining: 40.4s
493:	learn: 0.0999653	total: 39.3s	remaining: 40.3s
494:	learn: 0.0998747	total: 39.4s	remaining: 40.2s
495:	learn: 0.0997

636:	learn: 0.0877432	total: 50.1s	remaining: 28.6s
637:	learn: 0.0876859	total: 50.2s	remaining: 28.5s
638:	learn: 0.0876459	total: 50.3s	remaining: 28.4s
639:	learn: 0.0876062	total: 50.3s	remaining: 28.3s
640:	learn: 0.0875666	total: 50.4s	remaining: 28.2s
641:	learn: 0.0873303	total: 50.5s	remaining: 28.1s
642:	learn: 0.0872908	total: 50.5s	remaining: 28.1s
643:	learn: 0.0872509	total: 50.6s	remaining: 28s
644:	learn: 0.0871347	total: 50.7s	remaining: 27.9s
645:	learn: 0.0870590	total: 50.8s	remaining: 27.8s
646:	learn: 0.0868610	total: 50.9s	remaining: 27.8s
647:	learn: 0.0868217	total: 51s	remaining: 27.7s
648:	learn: 0.0867652	total: 51s	remaining: 27.6s
649:	learn: 0.0867238	total: 51.1s	remaining: 27.5s
650:	learn: 0.0866381	total: 51.2s	remaining: 27.4s
651:	learn: 0.0865188	total: 51.3s	remaining: 27.4s
652:	learn: 0.0864799	total: 51.3s	remaining: 27.3s
653:	learn: 0.0864412	total: 51.4s	remaining: 27.2s
654:	learn: 0.0864007	total: 51.5s	remaining: 27.1s
655:	learn: 0.0862

...

231:	learn: 0.1388941	total: 20.8s	remaining: 1m 8s
232:	learn: 0.1387120	total: 20.9s	remaining: 1m 8s
233:	learn: 0.1384996	total: 21s	remaining: 1m 8s
234:	learn: 0.1382956	total: 21s	remaining: 1m 8s
235:	learn: 0.1380446	total: 21.1s	remaining: 1m 8s
236:	learn: 0.1377982	total: 21.2s	remaining: 1m 8s
237:	learn: 0.1375178	total: 21.3s	remaining: 1m 8s
238:	learn: 0.1373536	total: 21.4s	remaining: 1m 8s
239:	learn: 0.1370490	total: 21.4s	remaining: 1m 7s
240:	learn: 0.1369281	total: 21.5s	remaining: 1m 7s
241:	learn: 0.1366408	total: 21.6s	remaining: 1m 7s
242:	learn: 0.1363691	total: 21.7s	remaining: 1m 7s
243:	learn: 0.1360973	total: 21.8s	remaining: 1m 7s
244:	learn: 0.1358796	total: 21.8s	remaining: 1m 7s
245:	learn: 0.1357257	total: 21.9s	remaining: 1m 7s
246:	learn: 0.1356028	total: 22s	remaining: 1m 6s
247:	learn: 0.1353832	total: 22.1s	remaining: 1m 6s
248:	learn: 0.1352265	total: 22.1s	remaining: 1m 6s
249:	learn: 0.1349143	total: 22.2s	remaining: 1m 6s
250:	learn: 0.1347

393:	learn: 0.1124063	total: 33.8s	remaining: 52s
394:	learn: 0.1121055	total: 33.9s	remaining: 51.9s
395:	learn: 0.1120059	total: 34s	remaining: 51.8s
396:	learn: 0.1117908	total: 34.1s	remaining: 51.7s
397:	learn: 0.1115917	total: 34.1s	remaining: 51.6s
398:	learn: 0.1114967	total: 34.2s	remaining: 51.5s
399:	learn: 0.1114145	total: 34.3s	remaining: 51.4s
400:	learn: 0.1112998	total: 34.3s	remaining: 51.3s
401:	learn: 0.1112395	total: 34.4s	remaining: 51.2s
402:	learn: 0.1111867	total: 34.5s	remaining: 51.1s
403:	learn: 0.1110145	total: 34.6s	remaining: 51s
404:	learn: 0.1109400	total: 34.6s	remaining: 50.9s
405:	learn: 0.1108176	total: 34.7s	remaining: 50.8s
406:	learn: 0.1104743	total: 34.8s	remaining: 50.7s
407:	learn: 0.1104072	total: 34.9s	remaining: 50.6s
408:	learn: 0.1102092	total: 35s	remaining: 50.6s
409:	learn: 0.1101588	total: 35.1s	remaining: 50.5s
410:	learn: 0.1099984	total: 35.1s	remaining: 50.4s
411:	learn: 0.1099269	total: 35.2s	remaining: 50.2s
412:	learn: 0.109756

554:	learn: 0.0965307	total: 45.9s	remaining: 36.8s
555:	learn: 0.0963827	total: 46s	remaining: 36.7s
556:	learn: 0.0963321	total: 46s	remaining: 36.6s
557:	learn: 0.0962836	total: 46.1s	remaining: 36.5s
558:	learn: 0.0962366	total: 46.2s	remaining: 36.4s
559:	learn: 0.0961902	total: 46.3s	remaining: 36.3s
560:	learn: 0.0961424	total: 46.3s	remaining: 36.3s
561:	learn: 0.0959972	total: 46.4s	remaining: 36.2s
562:	learn: 0.0959517	total: 46.5s	remaining: 36.1s
563:	learn: 0.0958714	total: 46.5s	remaining: 36s
564:	learn: 0.0956448	total: 46.6s	remaining: 35.9s
565:	learn: 0.0955093	total: 46.7s	remaining: 35.8s
566:	learn: 0.0954619	total: 46.8s	remaining: 35.7s
567:	learn: 0.0954144	total: 46.8s	remaining: 35.6s
568:	learn: 0.0953687	total: 46.9s	remaining: 35.5s
569:	learn: 0.0953225	total: 47s	remaining: 35.4s
570:	learn: 0.0951637	total: 47s	remaining: 35.3s
571:	learn: 0.0951021	total: 47.1s	remaining: 35.3s
572:	learn: 0.0950557	total: 47.2s	remaining: 35.2s
573:	learn: 0.0949686	

715:	learn: 0.0841295	total: 57.4s	remaining: 22.8s
716:	learn: 0.0840935	total: 57.5s	remaining: 22.7s
717:	learn: 0.0840574	total: 57.5s	remaining: 22.6s
718:	learn: 0.0839912	total: 57.6s	remaining: 22.5s
719:	learn: 0.0839536	total: 57.7s	remaining: 22.4s
720:	learn: 0.0839180	total: 57.8s	remaining: 22.3s
721:	learn: 0.0838248	total: 57.8s	remaining: 22.3s
722:	learn: 0.0836565	total: 57.9s	remaining: 22.2s
723:	learn: 0.0836194	total: 58s	remaining: 22.1s
724:	learn: 0.0835840	total: 58s	remaining: 22s
725:	learn: 0.0835488	total: 58.1s	remaining: 21.9s
726:	learn: 0.0835136	total: 58.2s	remaining: 21.8s
727:	learn: 0.0833223	total: 58.2s	remaining: 21.8s
728:	learn: 0.0831342	total: 58.3s	remaining: 21.7s
729:	learn: 0.0830672	total: 58.4s	remaining: 21.6s
730:	learn: 0.0830309	total: 58.4s	remaining: 21.5s
731:	learn: 0.0829959	total: 58.5s	remaining: 21.4s
732:	learn: 0.0828610	total: 58.6s	remaining: 21.3s
733:	learn: 0.0826735	total: 58.7s	remaining: 21.3s
734:	learn: 0.0826

875:	learn: 0.0740383	total: 1m 9s	remaining: 9.77s
876:	learn: 0.0740100	total: 1m 9s	remaining: 9.69s
877:	learn: 0.0738487	total: 1m 9s	remaining: 9.61s
878:	learn: 0.0738207	total: 1m 9s	remaining: 9.53s
879:	learn: 0.0737921	total: 1m 9s	remaining: 9.46s
880:	learn: 0.0737635	total: 1m 9s	remaining: 9.37s
881:	learn: 0.0737350	total: 1m 9s	remaining: 9.29s
882:	learn: 0.0736929	total: 1m 9s	remaining: 9.21s
883:	learn: 0.0735361	total: 1m 9s	remaining: 9.13s
884:	learn: 0.0734787	total: 1m 9s	remaining: 9.06s
885:	learn: 0.0734504	total: 1m 9s	remaining: 8.98s
886:	learn: 0.0734221	total: 1m 9s	remaining: 8.9s
887:	learn: 0.0733939	total: 1m 9s	remaining: 8.83s
888:	learn: 0.0733657	total: 1m 10s	remaining: 8.75s
889:	learn: 0.0732082	total: 1m 10s	remaining: 8.67s
890:	learn: 0.0730893	total: 1m 10s	remaining: 8.59s
891:	learn: 0.0730471	total: 1m 10s	remaining: 8.51s
892:	learn: 0.0729610	total: 1m 10s	remaining: 8.43s
893:	learn: 0.0728487	total: 1m 10s	remaining: 8.35s
894:	le